# Day 1

In [8]:
from itertools import pairwise, tee
from pathlib import Path

INPUTS = Path("aoc2021_inputs")

# With lazy iterator:
count = 0
with open(INPUTS / "day1.txt", "r") as f:
    for prev_, next_ in pairwise(map(int, f)):
        count += next_ > prev_
assert count == 1292

def triplewise(it):
    a, b, c = tee(it, 3)
    next(b); next(c); next(c)
    yield from zip(a, b, c)
    
def nwise(it, n):
    its = tee(it, n)
    for i, it in enumerate(its):
        for _ in range(i):
            next(it)
    yield from zip(*its)
    
def triplewise_(it):
    yield from nwise(it, 3)

count = 0
with open(INPUTS / "day1.txt", "r") as f:
    sum_prev = float("+inf")
    for v1, v2, v3 in triplewise_(map(int, f)):
        sum_ = v1 + v2 + v3
        count += sum_ > sum_prev
        sum_prev = sum_
assert count == 1262

# Day 2

In [2]:
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day2.txt")

with open(INPUT_PATH, "r") as f:
    moves = f.readlines()

h, v = 0, 0
for line in moves:
    move, size = line.split()

    if move == "forward":
        h += int(size)
    elif move == "up":
        v -= int(size)
    else:
        v += int(size)
        
print(h, v, h * v)
assert h * v == 1727835

1905 907 1727835


In [3]:
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day2.txt")

with open(INPUT_PATH, "r") as f:
    moves = f.readlines()
    
horiz, depth, aim = 0, 0, 0
for line in moves:
    move, size = line.split()
    size = int(size)
    
    if move == "forward":
        horiz += size
        depth += size * aim
    elif move == "down":
        aim += size
    else:
        aim -= size
        
print(horiz, depth, aim, horiz * depth)

1905 810499 907 1544000595


# Day 3

In [39]:
from collections import Counter
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day3.txt")

counter = Counter()
with open(INPUT_PATH, "r") as f:
    for line in f:
        counter += Counter(enumerate(line.strip()))

gamma, epsilon = 0, 0
total_bits = len(counter) // 2
for n in range(total_bits):
    if counter[(n, "1")] > counter[(n, "0")]:
        gamma |= 1 << (total_bits - n - 1)
    else:
        epsilon |= 1 << (total_bits - n - 1)
print(gamma * epsilon)

# --- part 2 ---

with open(INPUT_PATH, "r") as f:
    lines = f.readlines()

under_consideration = lines
prefix = ""
while len(under_consideration) > 1:
    c = Counter(line[:len(prefix) + 1] for line in under_consideration)
    c[prefix + "1"] += 0.5  # Force ending in "1" to win in case of draw.
    (prefix, count), = c.most_common(1)
    under_consideration = [line for line in under_consideration if line.startswith(prefix)]
oxygen = int(under_consideration[0], 2)
print(oxygen)

under_consideration = lines
prefix = ""
while len(under_consideration) > 1:
    c = Counter(line[:len(prefix) + 1] for line in under_consideration)
    c[prefix + "0"] -= 0.5  # Force ending in "0" to lose in case of draw
    _, (prefix, count) = c.most_common(2)
    under_consideration = [line for line in under_consideration if line.startswith(prefix)]
co2 = int(under_consideration[0], 2)
print(co2)

print(oxygen * co2)

749376
3871
613
2372923


# Day 4

In [3]:
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day4.txt")

with open(INPUT_PATH, "r") as f:
    nums = next(f).strip().split(",")  # Read order in which numbers are drawn.
    data = f.read().strip()

# Build dictionary storing the position of each number.
num_pos = dict(zip(nums, range(len(nums))))

# `boards` is a list of pairs (dirs, nums)
# `dirs` contains a list of all the possible winning directions for a board;
# `nums` is the set of all the numbers that show up in that board.
boards = []
for subdata in data.split("\n\n"):
    # Store the rows and columns in the same list.
    directions = []
    # Set to keep track of all the numbers the card contains.
    numbers = set()
    for row in subdata.split("\n"):
        directions.append(row.split())
        # Update the numbers with the ones of the current row.
        numbers |= {*directions[-1]}
    # Add columns to the directions list.
    bs = len(directions)
    directions += [
        [directions[r][c] for r in range(bs)] for c in range(bs)
    ]

    boards.append((directions, numbers))

# Go over each board and figure out when that board is done.
done_at = [
    min(max(num_pos[n] for n in dir_) for dir_ in dirs)
    for dirs, _ in boards
]

def score_board(winning_move_idx):
    _, board_nums = boards[done_at.index(winning_move_idx)]
    unmarked_numbers = map(int, board_nums - set(nums[:winning_move_idx + 1]))
    return sum(unmarked_numbers) * int(nums[winning_move_idx])

print(score_board(min(done_at))) # part 1
print(score_board(max(done_at))) # part 2

27027
36975


# Day 5

In [39]:
from collections import Counter
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day5.txt")

def parse_line(string):
    return [list(map(int, point.split(","))) for point in string.split(" -> ")]

def sign(x):
    return (x > 0) - (x < 0)

def generate_segment(left, right):
    (xl, yl), (xr, yr) = left, right
    delta = max(abs(xl - xr), abs(yl - yr))
    dx, dy = sign(xr - xl), sign(yr - yl)
    return [(xl + dx * d, yl + dy * d) for d in range(delta + 1)]

with open(INPUT_PATH, "r") as f:
    points = [parse_line(line.strip()) for line in f]

counter = Counter()
for left, right in points:
    (xl, yl), (xr, yr) = left, right
    if xl != xr and yl != yr:
        continue
    counter += Counter(generate_segment(left, right))

print(sum(cnt > 1 for _, cnt in counter.items()))

counter = Counter()
for left, right in points:
    counter += Counter(generate_segment(left, right))

print(sum(cnt > 1 for _, cnt in counter.items()))

7142
20012


In [23]:
generate_segment((3, 4), (3, 7))

[(3, 4), (3, 5), (3, 6), (3, 7)]

In [24]:
generate_segment((3, 4), (7, 4))

[(3, 4), (4, 4), (5, 4), (6, 4), (7, 4)]

In [25]:
generate_segment((3, 4), (3, 0))

[(3, 0), (3, 1), (3, 2), (3, 3), (3, 4)]

In [30]:
Counter(generate_segment((3, 4), (0, 4))) + Counter(generate_segment((3, 4), (3, 0)))

Counter({(0, 4): 1,
         (1, 4): 1,
         (2, 4): 1,
         (3, 4): 2,
         (3, 0): 1,
         (3, 1): 1,
         (3, 2): 1,
         (3, 3): 1})

In [33]:
acc = 0
for pnt in set(counter):
    if counter[pnt] > 1:
        acc += 1

In [34]:
acc

948565